In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from tensorflow.keras import layers, models, optimizers
import numpy as np

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Reshape and rescale the input data
#training data num samples reshaped to 32x32 pixels with 3 color channels
x_train = np.reshape(x_train, (50000, 32, 32, 3))
#testing data shaped to same with 10000 samples
x_test = np.reshape(x_test, (10000, 32, 32, 3))
y_train = np.reshape(y_train, (50000, 1))
y_test = np.reshape(y_test, (10000, 1))
#proccesing 32 images at a time
batch_size = 32
# normalzing by dividing by 255 ensuring each pizel is 0 to 1
x_train = tf.keras.layers.Rescaling(scale=1 / 255)(x_train)
x_test = tf.keras.layers.Rescaling(scale=1 / 255)(x_test)

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(1000).batch(batch_size, drop_remainder=True).prefetch(1)


def define_discriminator(in_shape=(32,32,3)):
  model = models.Sequential()
 # normal
  model.add(layers.Conv2D(64, (3,3), padding='same', input_shape=in_shape))
  model.add(layers.LeakyReLU(alpha=0.2))
 # downsample
  model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))
 # downsample
  model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))
 # downsample
  model.add(layers.Conv2D(256, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))
 # classifier
  model.add(layers.Flatten())
  model.add(layers.Dropout(0.4))
  model.add(layers.Dense(1, activation='sigmoid'))
 # compile model
  opt = optimizers.Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

def make_generator():
    model = models.Sequential()
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4 * 2
    model.add(layers.Dense(n_nodes, input_dim=100))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((4, 4, 512)))
    # upsample to 8x8
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    # output layer
    model.add(layers.Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

def buildGAN(generator, discriminator):
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Adjust learning rates
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Compile discriminator
discriminator = define_discriminator()


# Compile generator
generator = make_generator()
gan = buildGAN(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=generator_optimizer)

# Define display images function
def displayimages(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = images.squeeze(axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image)
        plt.axis("off")
    plt.show()

# Define a function to display generated images
def display_generated_images(generator, codings_size, n_images=10):
    noise = tf.random.normal(shape=[n_images, codings_size])
    generated_images = generator.predict(noise)
    displayimages(generated_images)


def training_gan(gan, dataset, batch_size, codings_size, n_epochs, display_freq=1):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1}/{n_epochs}")
        for i, (X_batch, _) in enumerate(dataset):
            # Phase 1 - Training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)

            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.train_on_batch(X_fake_and_real, y1)

            # Phase 2 - Training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            gan.train_on_batch(noise, y2)

        # Display generated images
        if (epoch + 1) % display_freq == 0:
            print("Generating and displaying images...")
            display_generated_images(generator, codings_size)
            print("Images displayed.")



codingSize = 100

# Train GAN
epochs = 30
training_gan(gan, dataset, batch_size, codingSize, epochs)


Epoch 1/30


KeyboardInterrupt: 